# Operating on Data in Pandas

Pandas 역시 NumPy의 ufunc 기능 활용을 통해 산술연산을 빠르게 수행할 수 있다. 

Pandas는 또한 unary 연산의 경우 *index*와  *column label*을 결과에서 보존하며 binary 연산의 경우 자동적으로 *indices* 정렬(align)하는 편의성을 제공한다. 
따라서 순수 NumPy array를 활용하는 것보다 실수의 가능성을 줄인다. 

## Ufuncs: Index Preservation



In [0]:
import pandas as pd
import numpy as np

In [2]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int64

In [3]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


아래 예와 같이, 연산이후 Pandas object가 리턴되며 index가 보존된다. 

In [4]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

In [5]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


## UFuncs: Index Alignment



### Index alignment in Series

아래 데이터는 미국의 면적(*area*) 및 인구(*population*) 기준 top-3 주 정보이다. 

In [0]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population')

다음과 같이 인구밀도를 구하는 경우, 결과를 예상해 보자.

In [7]:
population / area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

In [8]:
type (population/area)

pandas.core.series.Series

결과는 두 입력 array index들의 합집합(*union*)이다. 

In [9]:
area.index | population.index

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

아래 예도 마찬가지이다.

In [10]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

 이 때, NaN을 선호하지 않는다면 이진연산자 대신 fill_value 파라미터를 이용한 method 호출을 통해 바꿀 수도 있다. 

In [11]:
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### Index alignment in DataFrame

이러한 alignment 처리는 ``DataFrame``에서는 column과 index 둘 다 발생한다. 

In [12]:
X = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB'))
X

,A,B
0,1,11
1,5,1


In [13]:
Y = pd.DataFrame(rng.randint(0, 10, (3, 3)),
                 columns=list('BAC'))
Y

,B,A,C
0,4,0,9
1,5,8,0
2,9,2,6


In [14]:
X + Y

,A,B,C
0,1.0,15.0,NaN
1,13.0,6.0,NaN
2,NaN,NaN,NaN


NaN 대신 X의 평균값으로 missing value를 대체하여 계산할 수 있다. 

In [17]:
fill = X.stack().mean() # fill = 4.5
X.add(Y, fill_value=fill)

,A,B,C
0,1.0,15.0,13.5
1,13.0,6.0,4.5
2,6.5,13.5,10.5


아래 표는 Python operator와 해당 Pandas object method 리스트를 보여준다. 

| Python Operator | Pandas Method(s)                      |
|-----------------|---------------------------------------|
| ``+``           | ``add()``                             |
| ``-``           | ``sub()``, ``subtract()``             |
| ``*``           | ``mul()``, ``multiply()``             |
| ``/``           | ``truediv()``, ``div()``, ``divide()``|
| ``//``          | ``floordiv()``                        |
| ``%``           | ``mod()``                             |
| ``**``          | ``pow()``                             |


## Ufuncs: Operations Between DataFrame and Series

``DataFrame`` 과 ``Series`` 사이 연산에서도 index 와 column alignment 는 비슷하게 유지된다. 

In [18]:
A = rng.randint(10, size=(3, 4))
A

array([[3, 8, 2, 4],
       [2, 6, 4, 8],
       [6, 1, 3, 8]])

In [19]:
A - A[0]

array([[ 0,  0,  0,  0],
       [-1, -2,  2,  4],
       [ 3, -7,  1,  4]])

위 결과는 NumPy broadcasting rule에 따른 array 계산 결과이다. 

Pandas에서도 비슷하게 default로 row-wise 계산이 이뤄진다. 

In [20]:
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,-1,-2,2,4
2,3,-7,1,4


In [21]:
df.subtract(df.iloc[0], axis=1) # 위와 같은 결과이다.

,Q,R,S,T
0,0,0,0,0
1,-1,-2,2,4
2,3,-7,1,4


만약 column-wise 연산을 하고자 한다면 object method를 ``axis`` attribute 지정을 통해 실행할 수 있다.

In [23]:
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,-5,0,-6,-4
1,-4,0,-2,2
2,5,0,2,7


앞서 예제와 비슷하게 ``DataFrame``/``Series`` operation도 자동적으로 둘 사이의 index를 정렬한다. 

In [24]:
df

,Q,R,S,T
0,3,8,2,4
1,2,6,4,8
2,6,1,3,8


In [26]:
halfrow = df.iloc[0, ::2]
halfrow

Q    3
S    2
Name: 0, dtype: int64

In [0]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-1.0,NaN,2.0,NaN
2,3.0,NaN,1.0,NaN
